In this notebook, I'll illustrate how to fine-tune a Hugging Face model and generate text using transformers.

Transformers are a type of model architecture that has revolutionized NLP tasks. They use self-attention mechanisms to process input data in parallel, making them efficient for large datasets.



### Underlying Math

The mathematics behind the transformer model, particularly the self-attention mechanism. This is the core idea that powers generative models like GPT-2 and GPT-3.

**Self-Attention Mechanism**

The **self-attention mechanism** allows the model to attend to different parts of the input sequence. It works by computing three vectors for each token in the sequence:
- **Query (Q)**: Represents the token's search request.
- **Key (K)**: Represents the token's content.
- **Value (V)**: Holds the token's information.

These vectors are calculated by multiplying the input embeddings $X$ with learned weight matrices:
$
Q = X W_Q, \quad K = X W_K, \quad V = X W_V
$

where:
- $X$ is the input matrix of embeddings.
- $ W_Q, W_K, W_V$ are learnable weight matrices.

**Scaled Dot-Product Attention**

Once we have these vectors, the model computes how much attention each token should give to others by taking the **dot product** of the Query and Key vectors. This gives an initial attention score:

$
\text{Attention Score} = Q \cdot K^T
$

This score is then **scaled** by $ \sqrt{d_k} $, where $d_k$ is the dimension of the key vector. This scaling ensures that the dot product values aren’t too large, which could destabilize the training process:

$
\text{Scaled Score} = \frac{Q \cdot K^T}{\sqrt{d_k}}
$

The **softmax** function is applied to convert these scores into probabilities (attention weights), so that they sum to 1:

$
\text{Attention Weights} = \text{softmax}\left( \frac{Q \cdot K^T}{\sqrt{d_k}} \right)
$


These attention weights are used to take a **weighted sum** of the Value vectors, resulting in the **output** of the self-attention layer:

$
\text{Output} = \text{Attention Weights} \cdot V
$

**Multi-Head Attention**

To allow the model to focus on different relationships in the input simultaneously, **multi-head attention** is used. Instead of computing attention once, the model computes it multiple times in parallel, using different sets of learned weights for each head. The outputs of all attention heads are concatenated and projected into the final output:

$
Q_i = X W_Q^i, \quad K_i = X W_K^i, \quad V_i = X W_V^i
$

The multi-head attention output is:

$$
\text{Multi-Head Output} = \text{concat}(\text{head}_1, \text{head}_2, \dots, \text{head}_h) W_O
$$

where:
- $h$ is the number of attention heads.
- $W_O$ is a projection matrix that combines the attention heads.

**Positional Encoding**

Since transformers process tokens in parallel, they don’t inherently know the order of tokens. To address this, **positional encoding** is added to the input embeddings to provide order information. The positional encoding is computed using sine and cosine functions:

$
PE_{(i, 2k)} = \sin\left(\frac{i}{10000^{2k/d}}\right), \quad PE_{(i, 2k+1)} = \cos\left(\frac{i}{10000^{2k/d}}\right)
$

where:
- $ i$ is the position of the token in the sequence.
- $d$ is the dimensionality of the token embeddings.

These positional encodings are added to the input embeddings to provide the model with information about the relative position of tokens in the sequence.

**Why Self-Attention?**

Self-attention is crucial for allowing transformers to handle long-range dependencies in text. By processing all tokens at once and allowing the model to decide which tokens to focus on, it can capture relationships between tokens that are far apart in the sequence. This ability to "pay attention" to the important parts of the input is what makes transformers so powerful for **text generation** and other natural language tasks.


### Pyton code:

- Fine-tune a pre-trained model on your dataset.

- Use the fine-tuned model to generate text based on a given prompt.


Step 1: Setting Up Your Environment

Install Hugging Face Transformers library along with PyTorch or TensorFlow (depending on your preference).

In [ ]:
!pip install transformers torch  # For PyTorch

In [1]:
import torch

In [3]:
import transformers
print(transformers.__version__)


4.51.3


In [ ]:
!pip install --upgrade transformers


Step 2: Loading a Pre-trained Model

Hugging Face provides a wide range of pre-trained models. Here’s how to load a pre-trained model and tokenizer for text generation.


In [4]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load pre-trained model and tokenizer
model_name = "gpt2"  # You can also try "gpt2-medium", "gpt2-large", etc.
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Set the model to evaluation mode
model.eval()


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

Step 3: Generating Text

Now that you have the model and tokenizer, you can generate text. Here’s a simple example:

In [5]:
# Function to generate text
def generate_text(prompt, max_length=50):
    # Encode the input prompt
    input_ids = tokenizer.encode(prompt, return_tensors='pt')

    # Generate text
    with torch.no_grad():
        output = model.generate(input_ids, max_length=max_length, num_return_sequences=1)

    # Decode the generated text
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text

# Example usage
prompt = "The individual has always had to struggle to keep from being overwhelmed by the tribe. If you try it"
generated = generate_text(prompt)
print(generated)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


The individual has always had to struggle to keep from being overwhelmed by the tribe. If you try it, you'll be able to get through it.

"I think it's a very important thing to have a good relationship with your tribe.


Next: Fine-tuning a Model on a Dataset
